### Conver the TAT Questioning and Answering Dataset to dataframe with QA and its corresponding Paragraphs    
- Scraped TATQA dataset from the following address: (https://github.com/NExTplusplus/TAT-QA/tree/master)
- To fine-tune the QA model, convert this JSON files into dataframe with questions, answers, and the text that answere referred to

### 1.Import necessary libraries and datasets

In [85]:
# Import pandas and json to view json file as dataframe
import pandas as pd
import json

In [86]:
# Load the tatqa dataset in json format as tables
path = "/Users/takumakoide/Library/CloudStorage/" \
      "OneDrive-Personal/UChicago_MScA/ADSP34003_CAPSTONE II/Dataset/tatqa_dataset/"
file = "tatqa_dataset_dev.json"
file_path = path+file

with open(file_path, 'r') as file:
    data_list = json.load(file)

In [87]:
# Load another set of the tatqa dataset in json format as tables
file2 = "tatqa_dataset_train.json"
file2_path = path+file2

with open(file2_path, 'r') as file:
    data_list_2 = json.load(file)

In [88]:
# Load the third tatqa dataset in json format as tables
file3 = "tatqa_dataset_train2.json"
file3_path = path+file3

with open(file3_path, 'r') as file:
    data_list_3 = json.load(file)

### 2. Iterate through the JSON file and pick the necessary information from "paragraphs" and "questions" keys

In [79]:
# Initialize an empty list to store selected data
selected_data_list = []

# Iterate through each dictionary in the list
for item in data_list:
    # Check if "questions" key is present
    if 'questions' in item:
        # Iterate through each "questions" item
        for question_item in item['questions']:
            # Extract selected keys from "questions" item
            selected_data = {
                'uid': question_item.get('uid'),
                'order': question_item.get('order'),
                'question': question_item.get('question'),
                'answer': question_item.get('answer'),
                'answer_type': question_item.get('answer_type'),
                'answer_from': question_item.get('answer_from'),
                'rel_paragraphs': question_item.get('rel_paragraphs'),
                'req_comparison': question_item.get('req_comparison'),
                'related_text': None  # Placeholder for the "text" information
            }

            # Check if "paragraphs" key is present and "rel_paragraphs" is not empty
            if 'paragraphs' in item and selected_data['rel_paragraphs']:
                # Iterate over each element in the "rel_paragraphs" list
                for rel_paragraph in selected_data['rel_paragraphs']:
                    # Extract the "text" based on "rel_paragraphs" and "order"
                    texts = [paragraph_item['text'] for paragraph_item in item['paragraphs'] if
                             int(paragraph_item.get('order', 0)) == int(rel_paragraph.strip('[]'))]
                    
                    # Combine the texts into a single string and assign to the "related_text" key
                    selected_data['related_text'] = ' '.join(texts)

            # Append the selected data to the list
            selected_data_list.append(selected_data)

# Create a DataFrame from the selected data list
df_selected_data = pd.DataFrame(selected_data_list)

# Display the DataFrame
print("Selected Data DataFrame:")
df_selected_data.head()

Selected Data DataFrame:


,uid,order,question,answer,answer_type,answer_from,rel_paragraphs,req_comparison,related_text
0,23801627-ff77-4597-8d24-1c99e2452082,1,What is the company paid on a cost-plus type c...,[our allowable incurred costs plus a profit wh...,span,text,[2],False,"On a fixed-price type contract, we agree to pe..."
1,4960801d-277d-4f79-8eca-c4d0200fa9d6,2,What is the amount of total sales in 2019?,"[$1,496.5]",span,table-text,[2],False,"On a fixed-price type contract, we agree to pe..."
2,593c4388-5209-4462-8b83-b429c8612c25,3,What are the contract types?,"[fixed-price type, cost-plus type, time-and-ma...",multi-span,text,"[1, 2]",False,"On a fixed-price type contract, we agree to pe..."
3,f4142349-eb72-49eb-9a76-f3ccb1010cbc,4,In which year is the amount of total sales the...,[2019],span,table-text,[2],True,"On a fixed-price type contract, we agree to pe..."
4,eb787966-fa02-401f-bfaf-ccabf3828b23,5,What is the change in Other in 2019 from 2018?,-12.6,arithmetic,table-text,[2],False,"On a fixed-price type contract, we agree to pe..."


### 3. Filter the Dataframe to keep the Text-based QA 
- Also, count the number of words used in answer to ensure that it is concise.

In [89]:
# Function to count the number of words in a string
def count_words(text):
    words = text.split()
    return sum(word.isalpha() for word in words)

In [90]:
# Apply the function to the "answer" column and filter rows
filtered_df = df_selected_data[df_selected_data['answer'].apply(lambda x: count_words(str(x)) > 
                                                                          sum(c.isdigit() for c in str(x)))]
# filter the dataframe with answers reffering to table
filtered_df = filtered_df[filtered_df['answer_from'] != 'table']

# Add a new column "answer_count_words" to the DataFrame
filtered_df['answer_count_words'] = filtered_df['answer'].apply(lambda x: count_words(str(x)))


# Display the filtered DataFrame
print("Filtered DataFrame:")
filtered_df

Filtered DataFrame:


,uid,order,question,answer,answer_type,answer_from,rel_paragraphs,req_comparison,related_text,answer_count_words
0,23801627-ff77-4597-8d24-1c99e2452082,1,What is the company paid on a cost-plus type c...,[our allowable incurred costs plus a profit wh...,span,text,[2],False,"On a fixed-price type contract, we agree to pe...",25
6,86ae8d77-4dcd-4f82-baac-61c6a2551760,1,How is industry end market information presented?,[consistently with our internal management rep...,span,text,"[1, 2]",False,(1) Industry end market information is present...,13
12,682e2b7e-2b40-4849-8671-2b85dd3c7f10,1,How is the discount rate for domestic plans de...,[By comparison against the FTSE pension liabil...,span,text,[2],False,"For domestic plans, the discount rate was dete...",11
13,de34ea96-2a39-4cd5-b4a3-62b0f730103a,2,How is the discount rate for international pla...,[By comparison against country specific AA cor...,span,text,[2],False,"For domestic plans, the discount rate was dete...",11
19,bde0702e-2847-485b-be4a-fb037790bd59,2,Which countries does the group operate defined...,"[Germany, Ghana, India, Ireland, Italy, the UK...",span,text,[10],False,The Group operates defined benefit schemes in ...,3
...,...,...,...,...,...,...,...,...,...,...
1627,98d2b708-8948-4755-9407-e8c9c7a97883,2,Where does the company reports the intangible ...,[The Company reports intangible impairment cha...,span,text,[4],False,"In 2019, FCC licenses increased $15.6 million,...",16
1632,75716aca-67de-45ad-a624-3e95c050478d,1,What are the impacts of significant revisions ...,[materially affect our results of operations a...,span,text,[2],False,Use of the cost-to-cost or other similar metho...,20
1633,234fa3d2-2a72-4660-9bb6-fd82ff8da1a2,2,What could result in the 2019 operating income...,[if underlying assumptions were to change such...,span,text,[3],False,Products and services provided under long-term...,28
1638,033b1629-01f9-444b-961e-7ef875a8a915,1,What does in-game net revenues include?,[In-game net revenues primarily includes the n...,span,text,[4],False,(1) In-game net revenues primarily includes th...,17


The code returned 244 QA that is text-based and concise enough to fine tune our model.   
We will repeat this process with other available datasets.

### 4. Repeat the Data Conversion process to get enough QA sets

In [101]:
# Initialize an empty list to store selected data
selected_data_list2 = []

# Iterate through each dictionary in the list
for item in data_list_2:
    # Check if "questions" key is present
    if 'questions' in item:
        # Iterate through each "questions" item
        for question_item in item['questions']:
            # Extract selected keys from "questions" item
            selected_data = {
                'uid': question_item.get('uid'),
                'order': question_item.get('order'),
                'question': question_item.get('question'),
                'answer': question_item.get('answer'),
                'answer_type': question_item.get('answer_type'),
                'answer_from': question_item.get('answer_from'),
                'rel_paragraphs': question_item.get('rel_paragraphs'),
                'req_comparison': question_item.get('req_comparison'),
                'related_text': None  # Placeholder for the "text" information
            }

            # Check if "paragraphs" key is present and "rel_paragraphs" is not empty
            if 'paragraphs' in item and selected_data['rel_paragraphs']:
                # Iterate over each element in the "rel_paragraphs" list
                for rel_paragraph in selected_data['rel_paragraphs']:
                    # Extract the "text" based on "rel_paragraphs" and "order"
                    texts = [paragraph_item['text'] for paragraph_item in item['paragraphs'] if
                             int(paragraph_item.get('order', 0)) == int(rel_paragraph.strip('[]'))]
                    
                    # Combine the texts into a single string and assign to the "related_text" key
                    selected_data['related_text'] = ' '.join(texts)

            # Append the selected data to the list
            selected_data_list2.append(selected_data)

# Create a DataFrame from the selected data list
df_selected_data2 = pd.DataFrame(selected_data_list2)

# Display the DataFrame
print("Selected Data DataFrame:")
df_selected_data2.head(3)

Selected Data DataFrame:


,uid,order,question,answer,answer_type,answer_from,rel_paragraphs,req_comparison,related_text
0,9c569bea-e036-4c27-8e41-82c6efab1d31,1,What does the Weighted average actuarial assum...,"[Rate of inflation, Rate of increase in salari...",multi-span,table,[],False,None
1,83dcc128-9777-4ae2-ba55-50162ac43257,2,How much is the 2019 rate of inflation?,[2.9],span,table,[],False,None
2,0384152d-1f9f-4b12-85a9-2ca5e1c97682,3,How much is the 2018 rate of inflation?,[2.9],span,table,[],False,None


In [102]:
# Apply the function to the "answer" column and filter rows
filtered_df2 = df_selected_data2[df_selected_data2['answer'].apply(lambda x: count_words(str(x)) > 
                                                                          sum(c.isdigit() for c in str(x)))]
# filter the dataframe with answers reffering to table
filtered_df2 = filtered_df2[filtered_df2['answer_from'] != 'table']

# Add a new column "answer_count_words" to the DataFrame
filtered_df2['answer_count_words'] = filtered_df2['answer'].apply(lambda x: count_words(str(x)))


# Display the filtered DataFrame
print("Filtered DataFrame:")
filtered_df2

Filtered DataFrame:


,uid,order,question,answer,answer_type,answer_from,rel_paragraphs,req_comparison,related_text,answer_count_words
6,dd2a72ca-f65c-4673-9820-f7cefcb06ee4,1,How was internally developed software capitali...,[at cost less accumulated amortisation.],span,text,[7],False,Internally developed software is capitalised a...,3
18,d4919a15-b02b-4392-99ae-9eccd114e0ff,1,Why did the Operating revenues decreased from ...,[primarily due to decreased other operating re...,span,text,[2],False,Operating revenues. Operating revenues decreas...,15
19,05236c86-d05a-40d9-b15f-8eb197e2e158,2,Why did the Operating costs decreased from 201...,[primarily due to the decreased depreciation e...,span,text,[3],False,Operating costs. Operating costs decreased by ...,18
20,f14ec8e3-78ca-490c-afcd-fb53de9e3d0a,3,Why did the gross margin decreased from 2018 t...,[primarily due to an annual decline of 2.9% in...,span,text,[4],False,Gross profit and gross margin. Gross profit de...,9
24,d6a3a306-fb34-48dc-838e-3d109f60d685,1,What led to increase in the revenue of ADG on ...,[driven by an increase in volumes of approxima...,span,text,[1],False,"On a sequential basis, ADG revenues were up 3....",22
...,...,...,...,...,...,...,...,...,...,...
13194,97e273d5-dac0-415d-9742-ee6a43287dbf,4,What is the limitation of using total resource...,[Total resources expended for R&D is a non-GAA...,span,text,[7],False,Total resources expended for R&D serves to ill...,32
13197,35db303f-13aa-43d6-b157-82671b75a7b5,1,How is the cost of the Restricted Stock Units ...,[using the fair value of the Company’s Common ...,span,text,[3],False,The cost of RSUs is determined using the fair ...,10
13209,69b23654-bdc4-41ad-af90-5e60ffc0461c,1,What information is provided in Note 21?,[further information on fair value hierarchies.],span,text,[2],False,Please refer to note 21 “Financial Instruments...,4
13210,a05237d9-71e8-4043-b730-bddbf60484e5,2,What are the types of derivative financial ins...,"[Forward freight agreements, Bunker swaps]",multi-span,table-text,[1],False,NOTE 19 – DERIVATIVE FINANCIAL INSTRUMENTS,1


In [ ]:
# Confirm the output QA dataset after filtering to check the count of remaining QA pairs and 
# the lengths of answer word count
filtered_df2.describe()

,order,answer_count_words
count,2051.000000,2051.000000
mean,1.897123,14.566065
std,1.039065,11.010412
min,1.000000,1.000000
25%,1.000000,7.000000
50%,2.000000,12.000000
75%,2.000000,20.000000
max,7.000000,86.000000


We find that the number of available QA pairs is 2051 and their answer length is 14.57words and useful as a concise summary. 

In [104]:
# Initialize an empty list to store selected data
selected_data_list3 = []

# Iterate through each dictionary in the list in json file
for item in data_list_3:
    # Check if "questions" key is present in json file
    if 'questions' in item:
        # Iterate through each "questions" item  in json file
        for question_item in item['questions']:
            # Extract selected keys from "questions" item  in json file
            selected_data = {
                'uid': question_item.get('uid'),
                'order': question_item.get('order'),
                'question': question_item.get('question'),
                'answer': question_item.get('answer'),
                'answer_type': question_item.get('answer_type'),
                'answer_from': question_item.get('answer_from'),
                'rel_paragraphs': question_item.get('rel_paragraphs'),
                'req_comparison': question_item.get('req_comparison'),
                'related_text': None  # Placeholder for the "text" information
            }

            # Check if "paragraphs" key is present and "rel_paragraphs" is not empty
            if 'paragraphs' in item and selected_data['rel_paragraphs']:
                # Iterate over each element in the "rel_paragraphs" list
                for rel_paragraph in selected_data['rel_paragraphs']:
                    # Extract the "text" based on "rel_paragraphs" and "order"
                    texts = [paragraph_item['text'] for paragraph_item in item['paragraphs'] if
                             int(paragraph_item.get('order', 0)) == int(rel_paragraph.strip('[]'))]
                    
                    # Combine the texts into a single string and assign to the "related_text" key
                    selected_data['related_text'] = ' '.join(texts)

            # Append the selected data to the list for final retreival
            selected_data_list3.append(selected_data)

# Create a DataFrame from the selected data list
df_selected_data3 = pd.DataFrame(selected_data_list3)

# Display the DataFrame
print("Selected Data DataFrame:")
df_selected_data3.head(3)

Selected Data DataFrame:


,uid,order,question,answer,answer_type,answer_from,rel_paragraphs,req_comparison,related_text
0,9c569bea-e036-4c27-8e41-82c6efab1d31,1,What does the Weighted average actuarial assum...,"[Rate of inflation, Rate of increase in salari...",multi-span,table,[],False,None
1,83dcc128-9777-4ae2-ba55-50162ac43257,2,How much is the 2019 rate of inflation?,[2.9],span,table,[],False,None
2,0384152d-1f9f-4b12-85a9-2ca5e1c97682,3,How much is the 2018 rate of inflation?,[2.9],span,table,[],False,None


In [105]:
# Apply the function to the "answer" column and filter rows
filtered_df3 = df_selected_data3[df_selected_data3['answer'].apply(lambda x: count_words(str(x)) > 
                                                                          sum(c.isdigit() for c in str(x)))]
# Filter the dataframe with answers reffering to table
# This filtering is implemented because we want to allow RAG to be trained with text-format context only to match with the grounding data we use (articles)
filtered_df3 = filtered_df3[filtered_df3['answer_from'] != 'table']

# Add a new column "answer_count_words" to the DataFrame
# The purpose of this code is to measure the length of answer text to check the conciseness of the answers
filtered_df3['answer_count_words'] = filtered_df3['answer'].apply(lambda x: count_words(str(x)))


# Display the filtered DataFrame
print("Filtered DataFrame:")
filtered_df3

Filtered DataFrame:


,uid,order,question,answer,answer_type,answer_from,rel_paragraphs,req_comparison,related_text,answer_count_words
6,dd2a72ca-f65c-4673-9820-f7cefcb06ee4,1,How was internally developed software capitali...,[at cost less accumulated amortisation.],span,text,[7],False,Internally developed software is capitalised a...,3
18,d4919a15-b02b-4392-99ae-9eccd114e0ff,1,Why did the Operating revenues decreased from ...,[primarily due to decreased other operating re...,span,text,[2],False,Operating revenues. Operating revenues decreas...,15
19,05236c86-d05a-40d9-b15f-8eb197e2e158,2,Why did the Operating costs decreased from 201...,[primarily due to the decreased depreciation e...,span,text,[3],False,Operating costs. Operating costs decreased by ...,18
20,f14ec8e3-78ca-490c-afcd-fb53de9e3d0a,3,Why did the gross margin decreased from 2018 t...,[primarily due to an annual decline of 2.9% in...,span,text,[4],False,Gross profit and gross margin. Gross profit de...,9
24,d6a3a306-fb34-48dc-838e-3d109f60d685,1,What led to increase in the revenue of ADG on ...,[driven by an increase in volumes of approxima...,span,text,[1],False,"On a sequential basis, ADG revenues were up 3....",22
...,...,...,...,...,...,...,...,...,...,...
13194,97e273d5-dac0-415d-9742-ee6a43287dbf,4,What is the limitation of using total resource...,[Total resources expended for R&D is a non-GAA...,span,text,[7],False,Total resources expended for R&D serves to ill...,32
13197,35db303f-13aa-43d6-b157-82671b75a7b5,1,How is the cost of the Restricted Stock Units ...,[using the fair value of the Company’s Common ...,span,text,[3],False,The cost of RSUs is determined using the fair ...,10
13209,69b23654-bdc4-41ad-af90-5e60ffc0461c,1,What information is provided in Note 21?,[further information on fair value hierarchies.],span,text,[2],False,Please refer to note 21 “Financial Instruments...,4
13210,a05237d9-71e8-4043-b730-bddbf60484e5,2,What are the types of derivative financial ins...,"[Forward freight agreements, Bunker swaps]",multi-span,table-text,[1],False,NOTE 19 – DERIVATIVE FINANCIAL INSTRUMENTS,1


In [106]:
# Check if the answer word counts is extensive or not.
filtered_df3.describe()

,order,answer_count_words
count,2051.000000,2051.000000
mean,1.897123,14.566065
std,1.039065,11.010412
min,1.000000,1.000000
25%,1.000000,7.000000
50%,2.000000,12.000000
75%,2.000000,20.000000
max,7.000000,86.000000


Based on the output above, we can conclude that the filtered TATQA dataset's answer word count is on average 14.57 words, which is concise enough for users to refer the model output for time saving solutions.

### 5. Merge the QA Datasets together and Export File

In [107]:
# Concatenate the DataFrames vertically
concatenated_df = pd.concat([filtered_df, filtered_df2, filtered_df3], ignore_index=True)

# Display the concatenated DataFrame
print("Concatenated DataFrame:")
concatenated_df

Concatenated DataFrame:


,uid,order,question,answer,answer_type,answer_from,rel_paragraphs,req_comparison,related_text,answer_count_words
0,23801627-ff77-4597-8d24-1c99e2452082,1,What is the company paid on a cost-plus type c...,[our allowable incurred costs plus a profit wh...,span,text,[2],False,"On a fixed-price type contract, we agree to pe...",25
1,86ae8d77-4dcd-4f82-baac-61c6a2551760,1,How is industry end market information presented?,[consistently with our internal management rep...,span,text,"[1, 2]",False,(1) Industry end market information is present...,13
2,682e2b7e-2b40-4849-8671-2b85dd3c7f10,1,How is the discount rate for domestic plans de...,[By comparison against the FTSE pension liabil...,span,text,[2],False,"For domestic plans, the discount rate was dete...",11
3,de34ea96-2a39-4cd5-b4a3-62b0f730103a,2,How is the discount rate for international pla...,[By comparison against country specific AA cor...,span,text,[2],False,"For domestic plans, the discount rate was dete...",11
4,bde0702e-2847-485b-be4a-fb037790bd59,2,Which countries does the group operate defined...,"[Germany, Ghana, India, Ireland, Italy, the UK...",span,text,[10],False,The Group operates defined benefit schemes in ...,3
...,...,...,...,...,...,...,...,...,...,...
4341,97e273d5-dac0-415d-9742-ee6a43287dbf,4,What is the limitation of using total resource...,[Total resources expended for R&D is a non-GAA...,span,text,[7],False,Total resources expended for R&D serves to ill...,32
4342,35db303f-13aa-43d6-b157-82671b75a7b5,1,How is the cost of the Restricted Stock Units ...,[using the fair value of the Company’s Common ...,span,text,[3],False,The cost of RSUs is determined using the fair ...,10
4343,69b23654-bdc4-41ad-af90-5e60ffc0461c,1,What information is provided in Note 21?,[further information on fair value hierarchies.],span,text,[2],False,Please refer to note 21 “Financial Instruments...,4
4344,a05237d9-71e8-4043-b730-bddbf60484e5,2,What are the types of derivative financial ins...,"[Forward freight agreements, Bunker swaps]",multi-span,table-text,[1],False,NOTE 19 – DERIVATIVE FINANCIAL INSTRUMENTS,1


In [108]:
# Specify the file path for the CSV file
csv_file_name = 'tat_qa_concatenated.csv'

# Export the DataFrame to a CSV file 
concatenated_df.to_csv(path+csv_file_name, index=False)

print(f"DataFrame exported to {path+csv_file_name}") 

DataFrame exported to /Users/takumakoide/Library/CloudStorage/OneDrive-Personal/UChicago_MScA/ADSP34003_CAPSTONE II/Dataset/tatqa_dataset/tat_qa_concatenated.csv


### APPENDIX
TAT-QA (Tabular And Textual dataset for Question Answering) contains 16,552 questions associated with 2,757 hybrid contexts from real-world financial reports.